# | default_exp core

In [ ]:
# | hide
from bertopic import BERTopic
from bertopic.vectorizers import OnlineCountVectorizer
import dagshub
from datetime import datetime
import dill as pickle
import dvc.api
from hdbscan import HDBSCAN
from itertools import tee, islice
import joblib
import mlflow
from mlflow.models import infer_signature
import nbdev
from nbdev.showdoc import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import (
    CountVectorizer
    , TfidfTransformer
    , TfidfVectorizer
    , 
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from src.custom_sklearn_text_transformer_mlflow import CustomSKLearnAnalyzer
from src.custom_stanza_mlflow import CustomSKLearnWrapper
import src.dataframe_preprocessor as dfpp
import stanza
from tqdm import tqdm
from umap import UMAP

In [ ]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128'

# | export
def foo():
    pass

In [ ]:
# | hide
# this function allows us to get the experiment ID from an experiment name
def get_experiment_id(name):
    exp = mlflow.get_experiment_by_name(name)
    if exp is None:
      exp_id = mlflow.create_experiment(name)
      return exp_id
    return exp.experiment_id

In [ ]:
def custom_analyzer(step_list, stanza_pipeline, minNgramLength, maxNgramLength, lemmatize=True):
    lowered = " brk ".join(map(str, [step for step in step_list if step is not None])).lower()

    preproc = stanza_pipeline(lowered)
    
    if lemmatize:
        lemmad = " ".join(map(str,
                            [word.lemma
                            for sent in preproc.sentences 
                            for word in sent.words if (
                                word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "SCONJ", "PUNCT"]
                                and word is not None
                            )]
                        )
                    )
    else:
        lemmad = " ".join(map(str,
                            [word.text
                            for sent in preproc.sentences 
                            for word in sent.words if (
                                word is not None
                            )]
                        )
                    )
    # analyze each line of the input string seperately
    for ln in lemmad.split(' brk '):
        # tokenize the input string (customize the regex as desired)
        at_least_two_english_characters_whole_words = "(?u)\b[a-zA-Z]{2,}\b"
        terms = re.split(at_least_two_english_characters_whole_words, ln)

        # loop ngram creation for every number between min and max ngram length
        for ngramLength in range(minNgramLength, maxNgramLength+1):

            # find and return all ngrams
            # for ngram in zip(*[terms[i:] for i in range(3)]): 
                # <-- solution without a generator (works the same but has higher memory usage)
            for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]):   # <-- solution using a generator
                
                ngram = ' '.join(map(str, ngram))
                # yield ngram
                return str(ngram)


In [ ]:
# | hide
nbdev.nbdev_export()

# | Below this are blocks to use DagsHub with MLflow

In [ ]:
#@markdown Enter the username of your DAGsHub account:
DAGSHUB_USER_NAME = "AaronWChen" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
DAGSHUB_EMAIL = "awc33@cornell.edu" #@param {type:"string"}

#@markdown Enter the repo name 
DAGSHUB_REPO_NAME = "MeaLeon"

#@markdown Enter the name of the branch you are working on 
BRANCH = "MLF-1/start-custom-sklearn-mlflow-model"
dagshub.init(repo_name=DAGSHUB_REPO_NAME
             , repo_owner=DAGSHUB_USER_NAME)


Repository initialized!

## Starting DEV stage for One Hot Encoded model

In [ ]:
mlflow.set_tracking_uri(f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# starter idea for making an experiment name can be the git branch, but need more specificity
experiment_name = f"{DAGSHUB_EMAIL}/one-hot-encode"
mlflow_exp_id = get_experiment_id(experiment_name)

# define model location
# model_directory = "/tmp/sklearn_model"
model_directory = "../models/sklearn_model"

# Define the required artifacts associated with the saved custom pyfunc
# sklearn_path = model_directory + ""
sklearn_model_path = model_directory + "/python_model.pkl"
sklearn_transformer_path = model_directory + "/sklearn_transformer.pkl"
transformed_recipes_path = model_directory + "/transformed_recipes.pkl"

artifacts = {'sklearn_model': sklearn_model_path,
             'sklearn_transformer': sklearn_transformer_path,
             'transformed_recipes': transformed_recipes_path
             }


### Data Preparation

In [ ]:
# instantiate stanza pipeline
stanza.download('en')
nlp = stanza.Pipeline('en', 
                    depparse_batch_size=50, 
                    depparse_min_length_to_batch_separately=50,
                    verbose=True,
                    use_gpu=False, # set to true when on cloud/not on streaming computer
                    batch_size=100
                    )

# load raw data and preprocess/clean
data = dvc.api.read(
    path='../data/recipes-en-201706/epicurious-recipes_m2.json'
    , mode='r')
raw_df = pd.read_json(data)
print('\n')
print('--------------')
print('Raw Dataframe:', end='\n')
print(raw_df.head())
print(raw_df.shape)


2024-02-11 23:03:56 INFO: Downloading default packages for language: en (English) ...
2024-02-11 23:03:58 INFO: File exists: /home/awchen/stanza_resources/en/default.zip
2024-02-11 23:04:02 INFO: Finished downloading models and saved to /home/awchen/stanza_resources.
2024-02-11 23:04:02 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-02-11 23:04:03 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2024-02-11 23:04:03 INFO: Using device: cpu
2024-02-11 23:04:03 INFO: Loading: tokenize
2024-02-11 23:04:03 INFO: Loading: pos
2024-02-11 23:04:03 INFO: Loading: lemma
2024-02-11 23:04:03 INFO: Loading: constituency
2024-02-11 23:04:04 INFO: Loading: depparse
2024-02-11 23:04:04 INFO: Loading: sentiment
2024-02-11 23:04:04 INFO: Loading: ner
2024-02-11 23:04:04 INFO: Done loading processors!




--------------
Raw Dataframe:
                         id  \
0  54a2b6b019925f464b373351   
1  54a408a019925f464b3733bc   
2  54a408a26529d92b2c003631   
3  54a408a66529d92b2c003638   
4  54a408a719925f464b3733cc   

                                                 dek  \
0  How does fried chicken achieve No. 1 status? B...   
1                                Spinaci all'Ebraica   
2  This majestic, moist, and richly spiced honey ...   
3  The idea for this sandwich came to me when my ...   
4  In 1930, Simon Agranat, the chief justice of t...   

                                     hed                   pubDate  \
0            Pickle-Brined Fried Chicken  2014-08-19T04:00:00.000Z   
1                   Spinach Jewish Style  2008-09-09T04:00:00.000Z   
2                  New Year’s Honey Cake  2008-09-10T04:00:00.000Z   
3  The B.L.A.Bagel with Lox and Avocado  2008-09-08T04:00:00.000Z   
4        Shakshuka a la Doktor Shakshuka  2008-09-09T04:00:00.000Z   

                       

In [ ]:
# take sample and train/test split 
subset_df = raw_df.sample(n=100, random_state=45)
train_df, test_df = train_test_split(subset_df,test_size=0.5, random_state=45)

# pre_proc_df is cleaned dataframe
pre_proc_df = dfpp.preprocess_dataframe(train_df)
print('\n')
print('--------------')
print('Preprocessed Dataframe:', end='\n')
print(pre_proc_df.head())
print(pre_proc_df.shape)

# create subset for dev purposes
to_nlp_df = pre_proc_df
print('\n')
print('-' * 80)
print('Subset Dataframe:', end='\n')
print(to_nlp_df.head())
print(to_nlp_df.shape)



--------------
Preprocessed Dataframe:
                                                                        dek  \
id                                                                            
54a4270b19925f464b37c1dc                                                      
54a42cde19925f464b3809d2  Green chiles pickled in soy sauce and vinegar ...   
54a433036529d92b2c015de3  This soup features the flavors of India: aroma...   
54a451926529d92b2c01eda8                                                      
54a430876529d92b2c013e2b  Brown sugar and molasses are balanced by fresh...   

                                                                        hed  \
id                                                                            
54a4270b19925f464b37c1dc  Grilled Hearts of Romaine with Blue Cheese Vin...   
54a42cde19925f464b3809d2                              Soy-Pickled Jalapeños   
54a433036529d92b2c015de3  Curried Potato and Spinach Soup with Onion Sal...   
54a4519265

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_transformer_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': CustomSKLearnAnalyzer().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':3,
    'binary':True
}

# bertopic_params are a superset of cv_params
# bertopic_params = {
#     'top_n_words':20,
#     'min_topic_size':5,
#     'nr_topics':'auto',
#     'verbose':True,
#     'low_memory':True,
#     'calculate_probabilities':True
# }

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'OneHotEncoder'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_transformer_params)
# pipeline_params.update(bertopic_params)

signature = infer_signature(model_input=to_nlp_df['ingredients'],
                            )

with mlflow.start_run(experiment_id=mlflow_exp_id):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    # Will be useful in STAGING/Evaluation
    
    # LOG MODEL
    # Instantiate sklearn OneHotEncoder
    sklearn_transformer = CountVectorizer(**sklearn_transformer_params)

    print('\n')
    print('-' * 80)
    print('sklearn fit transform on ingredients:', end='\n')

    # Do fit transform on data
    response = sklearn_transformer.fit_transform(tqdm(to_nlp_df['ingredients'])) 
    
    transformed_recipe = pd.DataFrame(
            response.toarray(),
            columns=sklearn_transformer.get_feature_names_out(),
            index=to_nlp_df.index
    )

    print('\n')
    print('-' * 80)
    print('Transformed Data:', end='\n')
    print(transformed_recipe)
    
    # mlflow.pyfunc.save_model(
    #     path=model_directory,
    #     code_path=["../src/"],
    #     python_model=CustomSKLearnWrapper(),
    #     input_example=to_nlp_df['ingredients'][0],    
    #     artifacts=artifacts
    # )

     # joblib.dump(sklearn_transformer, sklearn_transformer_path)
    with open(sklearn_transformer_path, "wb") as fo:
        pickle.dump(sklearn_transformer, fo)
        # mlflow.log_artifact(sklearn_transformer_path,
        #                     artifact_path='sklearn_transformer')

    # joblib.dump(transformed_recipe, transformed_recipes_path)
    with open(transformed_recipes_path, "wb") as fo:
        pickle.dump(transformed_recipe, fo)
        # mlflow.log_artifact(transformed_recipes_path,
        #                     artifact_path='transformed_recipes')


    model_info = mlflow.pyfunc.log_model( 
        code_path=["../src/"],
        python_model=CustomSKLearnWrapper(),
        input_example=to_nlp_df['ingredients'][0],
        signature=signature,        
        artifact_path="sklearn_model",
        artifacts=artifacts
        ) 

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    
    



--------------------------------------------------------------------------------
sklearn fit transform on ingredients:


100%|██████████| 50/50 [01:36<00:00,  1.92s/it]




--------------------------------------------------------------------------------
Transformed Data:
                          cup chop onion  cup olive oil  cup sour cream  \
id                                                                        
54a4270b19925f464b37c1dc               0              1               0   
54a42cde19925f464b3809d2               0              0               0   
54a433036529d92b2c015de3               0              0               0   
54a451926529d92b2c01eda8               0              0               0   
54a430876529d92b2c013e2b               1              0               0   
54a453df6529d92b2c020687               0              0               1   
55b0e7116284773353bf4580               0              0               0   
54a42bab6529d92b2c00ffa7               0              0               0   
54a4748f19925f464b399ef2               0              1               0   
54a4356a19925f464b3875bb               0              1               0   

2024/02/11 23:05:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp9gtrkzjy/model, flavor: python_function), fall back to return ['cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


In [ ]:
response

<50x14 sparse matrix of type '<class 'numpy.int64'>'
	with 63 stored elements in Compressed Sparse Row format>

In [ ]:
test_predictor = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

2024/02/11 23:21:45 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


In [ ]:
# pre_proc_df is cleaned dataframe
pre_proc_test_df = dfpp.preprocess_dataframe(test_df)
print('\n')
print('--------------')
print('Preprocessed Dataframe:', end='\n')
print(pre_proc_test_df.head())
print(pre_proc_test_df.shape)

# create subset for dev purposes
to_nlp_test_df = pre_proc_test_df
print('\n')
print('-' * 80)
print('Subset Dataframe:', end='\n')
print(to_nlp_test_df.head())
print(to_nlp_test_df.shape)




--------------
Preprocessed Dataframe:
                                                                        dek  \
id                                                                            
54a418b26529d92b2c006395             Can be prepared in 45 minutes or less.   
54a4630719925f464b395a0c             Can be prepared in 45 minutes or less.   
54a4246f6529d92b2c00a770                                                      
54a469726529d92b2c0279a4  Easy and colorful, this side dish is nice with...   
54a4345019925f464b386748  Goes great with: Couscous flavored with choppe...   

                                                                        hed  \
id                                                                            
54a418b26529d92b2c006395  Braised Chicken with Mushrooms and Sun-Dried T...   
54a4630719925f464b395a0c                         Radishes with Chive Butter   
54a4246f6529d92b2c00a770                              Fried Blackberry Pies   
54a4697265

In [ ]:
test_predictor.predict(to_nlp_test_df['ingredients'])

ValueError: Shape of passed values is (1, 14), indices imply (49, 14)

In [ ]:
to_nlp_test_df['ingredients'].index

In [ ]:
to_nlp_test_df['ingredients'].values

In [ ]:
test_predictor.predict(to_nlp_test_df['ingredients'])

In [ ]:
test_predictor.signature.to_dict()

In [ ]:
model_info.signature.to_dict()

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# load dataframes from artifacts
# mlflow.artifacts.download_artifacts(
#     run_id=mlflow_run_id
# )

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# bertopic_params are a superset of cv_params
bertopic_params = {
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':'auto',
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(cv_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_stanza_ingreds_full_set_v1")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print('Raw Dataframe:', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print('Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    # to_nlp_df = pre_proc_df[0:50]
    # print('\n')
    # print('-' * 80)
    # print('Subset Dataframe:', end='\n')
    # print(to_nlp_df.head())
    # print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params,
    )

    def custom_analyzer(step_list, stanza_pipeline, minNgramLength, maxNgramLength):
            lowered = " brk ".join(map(str, [step for step in step_list if step is not None])).lower()

            preproc = stanza_pipeline(lowered)
            
            lemmad = " ".join(map(str,
                                [word.text
                                for sent in preproc.sentences 
                                for word in sent.words if (
                                    word is not None
                                )]
                            )
                        )
            
            # analyze each line of the input string seperately
            for ln in lemmad.split(' brk '):
                
                # tokenize the input string (customize the regex as desired)
                at_least_two_english_characters_whole_words = "(?u)\b[a-zA-Z]{2,}\b"
                terms = re.split(at_least_two_english_characters_whole_words, ln)

                # loop ngram creation for every number between min and max ngram length
                for ngramLength in range(minNgramLength, maxNgramLength+1):

                    # find and return all ngrams
                    # for ngram in zip(*[terms[i:] for i in range(3)]): 
                        # <-- solution without a generator (works the same but has higher memory usage)
                    for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]):   # <-- solution using a generator
                        
                        ngram = ' '.join(map(str, ngram))
                        # yield ngram
                        return str(ngram)

    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_ingreds = pre_proc_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print('Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # train on the recipes' steps
    topics, probs = topic_model.fit_transform(recipe_ingreds)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    # steps_vectorizer_model = CountVectorizer(**cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    steps_vectorizer_model = OnlineCountVectorizer(**cv_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print('BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print('BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')


### Attempt run with lighter weight configuration
#### This attempt will still use Stanza processing on the ingredients 

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_nlp_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# create sklearn pipeline as in BERTopic lightweight configuration
# pipe = make_pipeline(
#     TfidfVectorizer(**sklearn_nlp_params),
#     TruncatedSVD(100)
# )

# bertopic_params are a superset of cv_params
bertopic_params = {
    # 'embedding_model': TfidfVectorizer(**sklearn_nlp_params),
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':50,
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True,
    # 'min_cluster_size': 10 # Possibly only works if modifying individual HDBSCAN component of BERTopic
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_nlp_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_lightweight_stanza_ingreds_small_set_v1")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Raw Dataframe: ', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    to_nlp_df = pre_proc_df[0:100]
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Subset Dataframe:', end='\n')
    print(to_nlp_df.head())
    print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params
    )
    
    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_ingreds = to_nlp_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    # Create TF-IDF embeddings
    vectorizer = TfidfVectorizer(**sklearn_nlp_params)
    embeddings = vectorizer.fit_transform(recipe_ingreds)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # train on the recipes' ingredientss
    topics, probs = topic_model.fit_transform(recipe_ingreds, embeddings)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    sklearn_cv_params = {
        'strip_accents':"unicode",
        'lowercase':True,
        'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
        # 'min_df':10,
    }
    steps_vectorizer_model = CountVectorizer(**sklearn_cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    # steps_vectorizer_model = OnlineCountVectorizer(**sklearn_nlp_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representative Docs:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')


In [ ]:
topic_model.get_topic_info()['Representation']

In [ ]:
topic_model.get_topic_info()['Representation'][0]

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_nlp_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# create sklearn pipeline as in BERTopic lightweight configuration
# pipe = make_pipeline(
#     TfidfVectorizer(**sklearn_nlp_params),
#     TruncatedSVD(100)
# )

# bertopic_params are a superset of cv_params
bertopic_params = {
    # 'embedding_model': TfidfVectorizer(**sklearn_nlp_params),
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':50,
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True,
    # 'min_cluster_size': 10 # Possibly only works if modifying individual HDBSCAN component of BERTopic
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_nlp_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_lightweight_stanza_ingreds_small_set_v1")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Raw Dataframe: ', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    to_nlp_df = pre_proc_df[0:100]
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Subset Dataframe:', end='\n')
    print(to_nlp_df.head())
    print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params
    )
    
    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4}
    
    recipe_ingreds = to_nlp_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    # Create TF-IDF embeddings
    vectorizer = TfidfVectorizer(**sklearn_nlp_params)
    embeddings = vectorizer.fit_transform(recipe_ingreds)

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # train on the recipes' ingredientss
    topics, probs = topic_model.fit_transform(recipe_ingreds, embeddings)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    sklearn_cv_params = {
        'strip_accents':"unicode",
        'lowercase':True,
        # 'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
        # 'min_df':10,
    }
    steps_vectorizer_model = CountVectorizer(**sklearn_cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    # steps_vectorizer_model = OnlineCountVectorizer(**sklearn_nlp_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=steps_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representative Docs:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')


In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_nlp_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# create sklearn pipeline as in BERTopic lightweight configuration
# pipe = make_pipeline(
#     TfidfVectorizer(**sklearn_nlp_params),
#     TruncatedSVD(100)
# )

# bertopic_params are a superset of cv_params
bertopic_params = {
    # 'embedding_model': TfidfVectorizer(**sklearn_nlp_params),
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':50,
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True,
    # 'min_cluster_size': 10 # Possibly only works if modifying individual HDBSCAN component of BERTopic
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_nlp_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_lightweight_stanza_ingreds_small_set_v1.01")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Raw Dataframe: ', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    to_nlp_df = pre_proc_df[0:100]
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Subset Dataframe:', end='\n')
    print(to_nlp_df.head())
    print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params
    )
    
    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4
                       , 'lemmatize': True}
    
    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    recipe_ingreds = to_nlp_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    print([ingred for ingred in recipe_ingreds])

    # Create TF-IDF embeddings
    vectorizer = TfidfVectorizer(**sklearn_nlp_params)
    embeddings = vectorizer.fit_transform(tqdm(recipe_ingreds))

    # train on the recipes' ingredientss
    topics, probs = topic_model.fit_transform(recipe_ingreds, embeddings)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    sklearn_cv_params = {
        # 'strip_accents':"unicode",
        # 'lowercase':True,
        # 'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
        # 'min_df':10,
        'token_pattern': r"(?u)\b[a-zA-Z]{2,}\b"
    }
    ingreds_vectorizer_model = CountVectorizer(**sklearn_cv_params)

    # May need to use BERTopic's OnlineCountVectorizer
    # steps_vectorizer_model = OnlineCountVectorizer(**sklearn_nlp_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=ingreds_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representative Docs:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_small_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_small_set_df.json',
                        artifact_path='bertopic_models')


In [ ]:
to_nlp_df['ingredients'][0]

In [ ]:
test_recipe_ingreds = to_nlp_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)

i think i should start leaving out units/including stopwords again since i'm not using Stanza's deep learning

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_nlp_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

# bertopic_params are a superset of cv_params
bertopic_params = {
    'top_n_words':20,
    'min_topic_size':10,
    'nr_topics':50,
    'verbose':True,
    'low_memory':True,
    'calculate_probabilities':True,
    # 'min_cluster_size': 10 # Possibly only works if modifying individual HDBSCAN component of BERTopic
}

# update bertopic_params to include cv_params
# bertopic_params.update(cv_params)

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'TfidfVectorizer'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_nlp_params)
pipeline_params.update(bertopic_params)

with mlflow.start_run(experiment_id=get_experiment_id(f"{DAGSHUB_EMAIL}/bertopic_lightweight_stanza_ingreds_full_set_v1.00")):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    
    # load raw data and preprocess/clean
    data = dvc.api.read(
           path='../data/recipes-en-201706/epicurious-recipes_m2.json'
           , mode='r')
    raw_df = pd.read_json(data)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Raw Dataframe: ', end='\n')
    print(raw_df.head())
    print(raw_df.shape)

    # pre_proc_df is cleaned dataframe
    pre_proc_df = dfpp.preprocess_dataframe(raw_df)
    print('\n')
    print('--------------')
    print(f'{datetime.now()}, Preprocessed Dataframe:', end='\n')
    print(pre_proc_df.head())
    print(pre_proc_df.shape)


    # pre_proc_df = pd.read_json(
    #     mlflow.artifacts.download_artifacts(
    #         run_id=mlflow_run_id,
    #         artifact_path='artifacts/preprocessed_dataframes/preprocessed_dataframe.json',
    #         # tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow'
    #     )
    # )
    # print('\n')
    # print('-' * 80)
    # print('Preprocessed Dataframe:', end='\n')
    # print(pre_proc_df.head())
    # print(pre_proc_df.shape)

    # create subset for dev purposes
    # to_nlp_df = pre_proc_df[0:100]
    # print('\n')
    # print('-' * 80)
    # print(f'{datetime.now()}, Subset Dataframe:', end='\n')
    # print(to_nlp_df.head())
    # print(to_nlp_df.shape)

    # LOG MODEL
    # Instantiate BERTopic
    topic_model = BERTopic(
        **bertopic_params
    )
    
    analyzer_kwargs = {'stanza_pipeline': nlp
                       , 'minNgramLength': 1
                       , 'maxNgramLength': 4
                       , 'lemmatize': True}
    
    recipe_ingreds = pre_proc_df["ingredients"].apply(custom_analyzer, **analyzer_kwargs)
    
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    print(recipe_ingreds)

    # Create TF-IDF embeddings
    vectorizer = TfidfVectorizer(**sklearn_nlp_params)
    embeddings = vectorizer.fit_transform(tqdm(recipe_ingreds))

    # recipe_steps = "".join(str(to_nlp_df["prepSteps"].apply(StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4))))
    # print('\n')
    # print('-' * 80)
    # print(f'{datetime.now()}, Recipe ingredients:', end='\n')
    # print(recipe_ingreds)

    # train on the recipes' ingredientss
    topics, probs = topic_model.fit_transform(recipe_ingreds, embeddings)

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    # Instantiate sklearn CountVectorizer
    sklearn_cv_params = {
        # 'strip_accents':"unicode",
        # 'lowercase':True,
        'token_pattern': r"(?u)\b[a-zA-Z]{2,}\b"
    }
    ingreds_vectorizer_model = CountVectorizer(**sklearn_cv_params)

    # Do fit transform on data
    # steps_test_tfidf_transform = steps_tfidf_vectorizer_model.fit_transform(tqdm(to_nlp_df["steps"]))
    topic_model.update_topics(
        recipe_ingreds
        , vectorizer_model=ingreds_vectorizer_model
    )

    # Display topic model results
    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Dataframe:', end='\n')
    print(topic_model.get_topic_info())

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representations:', end='\n')
    print(topic_model.get_topic_info()['Representation'])

    print('\n')
    print('-' * 80)
    print(f'{datetime.now()}, BERTopic Model Representative Docs:', end='\n')
    print(topic_model.get_topic_info()['Representative_Docs'])

    # Save and log the topic model dataframe
    topic_model.get_topic_info().to_json('../data/processed/bertopic_model_ingreds_full_set_df.json')
    mlflow.log_artifact('../data/processed/bertopic_model_ingreds_full_set_df.json',
                        artifact_path='bertopic_models')


In [ ]:
# try splitting among CPU and GPU. Try Stanza on CPU due to its memory usage
nlp2 = stanza.Pipeline('en', use_gpu=False)

